# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:

try:
    session.execute("""CREATE KEYSPACE  IF NOT EXISTS udacity\
                    WITH REPLICATION =
                    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Query#1 :Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Table name:- Table1

Primary Key :- sessionId,itemInSession

Partition Key :-sessionId,

Clustering Key :- itemInSession

Code flow :-SELECT and INSERT  data for  all columns in the begining and SELECT only required columns in the final Phase.

In [8]:
## Create table table1
query="CREATE TABLE IF NOT EXISTS table1 "
query=query+"(sessionId int,itemInSession int,artist text,firstName text,gender text,lastName text,length float,level text,location text,song text,userId int, PRIMARY KEY(sessionId,itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
#Insert data into table1 for each row in the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO table1 (sessionId ,itemInSession ,artist ,firstName ,gender ,lastName ,length ,level ,location ,song ,userId )"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        ## Assigning  column element 
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[1], line[2],line[4],float(line[5]), line[6],line[7],line[9],int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# SELECT statement to verify the data was entered into the table
#Select only required columns from the table
query = "select artist,song,length from table1 WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Query#2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

Table name:- Table2

Primary Key :- userId, sessionId

Partition Key :-userId, sessionId

Clustering Key :- itemInSession

Code flow :-SELECT and INSERT  data for  all columns in the begining and SELECT only required columns in the final Phase.

In [11]:
# Query 2: Create table table2
query="CREATE TABLE IF NOT EXISTS table2 "
query=query+"(sessionId int,itemInSession int,artist text,firstName text,gender text,lastName text,length float,level text,location text,song text,userId int,PRIMARY KEY((userid,sessionId),itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    
                    

In [12]:
#Insert data into table2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO table2 (sessionId ,itemInSession ,artist ,firstName ,gender ,lastName ,length ,level ,location ,song ,userId )"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        #Assigning column element .
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[1], line[2],line[4],float(line[5]), line[6],line[7],line[9],int(line[10])))

In [13]:
#Adding in the SELECT statement to verify the data was entered into the table
#select only required columns from table2
query = "select artist ,song,firstName ,lastName from table2 WHERE userid=10 and sessionId=182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song,row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


Query#3  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


Table name:- Table3

Primary Key :- song,userId

Partition Key :-song

Clustering Key :- userId

Code flow :-SELECT and INSERT  data for  all columns in the begining and SELECT only required columns in the final Phase.

In [14]:
#Query 3: create table table3
query="CREATE TABLE IF NOT EXISTS table3 "
query=query+"(sessionId int,itemInSession int,artist text,firstName text,gender text,lastName text,length float,level text,location text,song text,userId int,PRIMARY KEY(song,userid))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                                   
                    

In [15]:
#Insert into table3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO table3 (sessionId ,itemInSession ,artist ,firstName ,gender ,lastName ,length ,level ,location ,song ,userId )"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assigning column element 
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[1], line[2],line[4],float(line[5]), line[6],line[7],line[9],int(line[10])))

In [16]:
#Adding in the SELECT statement to verify the data was entered into the table
#Select only required columns from table 3
query = "select firstName ,lastName from table3 WHERE song='All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
# Drop the table before closing out the sessions
session.execute("""DROP TABLE table1""")
session.execute("""DROP TABLE table2""")
session.execute("""DROP TABLE table3""")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()